# Actividad 2 Analisis de Informacion No Estructurada

# - Javier Gilabert Sabater

1. LeaelcontenidodelficherocsvenunDataFrame.
2. Realiceelpre-procesamientoqueconsiderenecesario.Puede
utilizar funciones de la librería NLTK o spaCy, a su voluntad. Recomendamos una escritura modular del código, para poder hacer pruebas posteriormente, y contestar a las preguntas del punto 6.
3. Convierta el corpus de documentos en una matriz TF-idf.
4. Dividaenunsubconjuntodeentrenamientoyotrodeevaluación.
5. Llegadosaestepunto,realicemodelosdeentrenamientoalmenos
con algoritmos de clasificador bayesiano ingenuo, máquinas SVM y un modelo basado en árbol de decisión. Obtenga resultados de accuracy de la clasificación, así como las matrices de confusión para los tres modelos.
6. Contestealassiguientespreguntasbasándoteenevidenciasde código. ¿Tiene influencia en el resultado final el número máximo de features a utilizar? Prueba al menos dos números de features diferentes para los tres algoritmos y mide los resultados. ¿Modifica el resultado si no se eliminan las stop words? Pruébalo para los tres algoritmos y mide los resultados.
7. Imagínesequeesteentregableesunalaborquelehansolicitadoen un entorno profesional, y que tiene que entregar esta documentación para comentar lo que ha descubierto (datos de entrada, rendimiento de los modelos, o cualquier descubrimiento que pueda ser importante). Comente los resultados obtenidos.


## 1. Lea el contenido del fichero csv en un DataFrame.

In [22]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [23]:
df = pd.read_csv("spam.csv", sep=',')
df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


## 2. Realice el pre-procesamiento que considere necesario. Puede utilizar funciones de la librería NLTK o spaCy, a su voluntad. Recomendamos una escritura modular del código, para poder hacer pruebas posteriormente, y contestar a las preguntas del punto 6.

In [24]:
corpus = df.text.to_list()

def limpiar_texto(texto):
    if not isinstance(texto, str):
        raise ValueError("El argumento 'texto' debe ser una cadena de texto.")
    
    # Eliminar caracteres especiales y números
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)
    
    # Convertir a minúsculas
    texto = texto.lower()
    
    # Tokenizar el texto
    tokens = word_tokenize(texto)
    
    # Eliminar stop words
    stop_words = set(stopwords.words('english'))
    tokens_filtrados = [word for word in tokens if word not in stop_words]
    
    # Unir tokens nuevamente en un string
    texto_limpiado = ' '.join(tokens_filtrados)
    
    return texto_limpiado

def limpiar_corpus(corpus):
    corpus_limpiado = []
    
    for email in corpus:
        try:
            email_limpiado = limpiar_texto(email)
            corpus_limpiado.append(email_limpiado)
        except ValueError as e:
            print(f"Error: {e}")
            corpus_limpiado.append('error')
    
    return corpus_limpiado

# Llamar a la función para limpiar el corpus
corpus_limpiado = limpiar_corpus(corpus)

In [25]:
df['corpus_limpiado'] = df['text'].apply(lambda x: limpiar_texto(x))
df

,label,text,corpus_limpiado
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,nd time tried contact u u pound prize claim ea...
5568,ham,Will Ì_ b going to esplanade fr home?,b going esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...",pity mood soany suggestions
5570,ham,The guy did some bitching but I acted like i'd...,guy bitching acted like id interested buying s...


## 3. Convierta el corpus de documentos en una matriz TF-idf.

In [28]:
# Crear una instancia de TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Ajustar el vectorizador a los argumentos preprocesados y transformar los datos
X_tfidf = tfidf_vectorizer.fit_transform(corpus_limpiado)

# Convertir la matriz X_tfidf a un DataFrame de pandas
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=corpus_limpiado)

# df_tfidf ahora contiene la representación TF-IDF en formato DataFrame
df_tfidf

,aa,aah,aaniye,aaooooright,aathilove,aathiwhere,ab,abbey,abdomen,abeg,...,zeros,zf,zhong,zindgi,zoe,zogtorius,zoom,zouk,zs,zyada
go jurong point crazy available bugis n great world la e buffet cine got amore wat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ok lar joking wif u oni,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
free entry wkly comp win fa cup final tkts st may text fa receive entry questionstd txt ratetcs apply overs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u dun say early hor u c already say,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nah dont think goes usf lives around though,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nd time tried contact u u pound prize claim easy call p per minute btnationalrate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
b going esplanade fr home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pity mood soany suggestions,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
guy bitching acted like id interested buying something else next week gave us free,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4. Divida en un subconjunto de entrenamiento y otro de evaluación.

In [30]:
random_num = 100
train_df, test_df = train_test_split(df, test_size=0.3, random_state=random_num)

## 5. Llegados a este punto, realice modelos de entrenamiento al menos con algoritmos de clasificador bayesiano ingenuo, máquinas SVM y un modelo basado en árbol de decisión. Obtenga resultados de accuracy de la clasificación, así como las matrices de confusión para los tres modelos.